<a href="https://colab.research.google.com/github/Azuremis/make_your_first_gan_with_pytorch/blob/master/mnist_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn

In [0]:
# neural network class
class Classifier(nn.Module):

  def __init__(self):
    # initialise parent pytorch class
    super().__init__()

    # setup neural network architecture
    self.model = nn.Sequential(
        nn.Linear( 784, 200), #  fully connected mapping from 784 nodes to 200 nodes
        nn.Sigmoid(),  # apply sigmoid to ouput of 200 nodes
        nn.Linear(200, 10),  # maps 200 nodes to 10 nodes
        nn.Sigmoid()  # apply sigmoid to output of 10 nodes to get final output
    )

    # setup loss function
    self.loss_function = torch.nn.MSELoss()

    # setup optimiser using simple stochastic gradient descent
    self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)

    # setup forward method for passing information through network
    def forward(self, inputs):
      # run the model
      return self.model(inputs)

    def train(self, inputs, targets):
      # calculate nn outputs
      outputs = self.forward(inputs)

      # calculate loss
      loss = self.loss_function(outputs, targets)

      # process nn updates
      self.optimiser().zero_grad() # set gradients to zero
      loss.backward() # calculate gradients via backward pass
      self.optimiser.step() # update nn weights using gradients